In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import bs4 as bs
from urllib.request import Request, urlopen
import pandas as pd
import time
import os
import requests
from IPython import embed
import re
import urllib.request

from datetime import datetime

website = 'https://www.pikalytics.com/pokedex/ss'

The purpose of the following two functions is to either download the html of a website page, or to open the website using selenium. In the case of websites which require dynamically generated content (without the url change. Javascript for example), selenium is required. For pikalytics, there are a lot of javascript interactions (one single url with a lot of different information depending on where we click), which means we need to open the page with selenium, which is a headless browser.

NOTE: The directory for "geckodriver" must be specified to the location of the file within the submission folder to allow the scraper to work correctly.

In [2]:
def fetch(page, addition=''):
    return bs.BeautifulSoup(urlopen(Request(page + addition,
            headers={'User-Agent': 'Opera/9.80 (X11; Linux i686; Ub'\
                     'untu/14.10) Presto/2.12.388 Version/12.16'})).read(), 'lxml')

def fetch_sel(page, headless = True):
	options = webdriver.FirefoxOptions()
	if headless:
		options.add_argument('-headless')
	direct = os.path.dirname(__file__)
	exe_path = os.path.join(direct, 'geckodriver')
	exe_path = '/Users/ajkea/Documents/geckodriver'
	driver = webdriver.Firefox(executable_path=exe_path,options=options)
	driver.get(page)
	
	html = driver.page_source
	return bs.BeautifulSoup(html,'lxml'), driver

The following function is the main scraping function. Here are the steps to get the data.

1. Create empty dataframes, and list of formats (used later)
2. The main scraping loop.
    1. Loop through every section (league). Here, we can't change the url to get to the desited page. We need to select the correct option from the dropdown menu.
    2. Once the section has been selected, we loop through the top 20 pokemons, clicking on each of them.
    3. Once we have clicked on a pokemon, the stats/items/abilities page appear. This is where we gather the information. This is done by finding the id of *moves_wrapper* for example and gathering the move/% pair.
    4. The information is then added to the full pandas dataframe.
3. Repeart 3.1 for the following sections

In [1]:
def get_pikalytics_stats(website, top_n=20):
	''' Gets all stats, moves, items for top 20 pokemons'''
	result_page, driver = fetch_sel(f'{website}')
	time.sleep(5)
	dropdown = driver.find_element_by_id("format_dd")
	options = dropdown.find_elements_by_tag_name('option')
	scrape_names = ['Showdown VGC 2021 (0+)','OverUsed (0+)','Ubers','UnderUsed','RarelyUsed','NeverUsed',
					'PU','VGC 2019 Ultra Series','VGC 2018','VGC 2017','Gen 7 OverUsed']
	
	df = pd.DataFrame(columns = ['section','poke_name','perc_use','url','stats','moves','items'])

	for section in [x.text for x in options]:
		if section in scrape_names:
			print(section)
			dropdown = driver.find_element_by_id("format_dd")
			selector = Select(dropdown)
			selector.select_by_visible_text(section)
			time.sleep(3)
			a = driver.find_element_by_xpath('//*[@id="min_list"]')
			len_a = len(a.find_elements_by_tag_name('a'))
			for i in range(1,min(top_n,len_a)+1):
				poke_box = driver.find_element_by_xpath(f'//*[@id="min_list"]/a[{i}]')
				url_ = poke_box.get_attribute('href')
				poke_name = poke_box.get_attribute('data-name')
				perc_val = float(driver.find_element_by_xpath(f'//*[@id="min_list"]/a[{i}]/span').text.strip().replace('%',''))/100
				
				bs4_text = fetch(url_)
				stats_ = bs4_text.find('div',{'id':'bstats_wrapper'})
				stats_names = stats_.find_all('div',{'style':'display:inline-block;width:30px;text-align: left;'})
				stats_vals = stats_.find_all('div',{'style':'display:inline-block;vertical-align: middle;margin-left: 20px;'})
				stats = ''
				for idx, v in enumerate(stats_names):
					stat = f'{stats_names[idx].text}:{stats_vals[idx].text};'
					stats += stat

				moves_ = bs4_text.find('div',{'id':'moves_wrapper'})
				move_list = ''
				for move in moves_.find_all('div',{'class':'pokedex-move-entry-new'}):
					move_name = move.find('div',{'style':'margin-left:10px;display:inline-block;'}).text
					move_perc = move.find('div',{'style':'display:inline-block;float:right;'}).text
					move_list += f"{move_name}:{move_perc};"

				items_ = bs4_text.find('div',{'id':'items_wrapper'})
				items_list = ''
				for item in items_.find_all('div',{'class':'pokedex-move-entry-new'}):
					item_name = item.find('div',{'style':'display:inline-block;'}).text
					item_perc = item.find('div',{'style':'display:inline-block;float:right;'}).text
					items_list += f"{item_name}:{item_perc};"

				df = df.append({'section':section,'poke_name':poke_name,'perc_use':perc_val,'url':url_,'stats':stats,
									'moves':move_list,'items':items_list}, ignore_index=True)


	df.to_csv('pikalytics.csv')

In [ ]:
get_pikalytics_stats(website)